<a href="https://colab.research.google.com/github/adarsh415/Tensorflow2.x/blob/master/CIFAR_VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow.keras import models, metrics, optimizers, layers, datasets, regularizers
import numpy as np

TensorFlow 2.x selected.


In [0]:
tf.__version__

'2.1.0'

In [0]:
def load_data():
    (xtrain, ytrain), (xtest, ytest) = datasets.cifar10.load_data()
    print('train shape:', xtrain.shape, xtest.shape)
    print('test shape:', xtest.shape, ytest.shape)
    xtrain = xtrain.astype(np.float32)/255.0
    xtest = xtest.astype(np.float32)/255.0
    trainDS = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
    trainDS = trainDS.shuffle(50000).batch(64)
    testDS = tf.data.Dataset.from_tensor_slices((xtest, ytest))
    testDS = testDS.batch(128)
    
    return trainDS, testDS

In [0]:
config = {
    "filters" : [64,128,256,512],
    "conv" : 3,
    "num_classes": 10,
    "input_shape": (32,32,3)
}

In [0]:
class VGG16(models.Model):
    
    def __init__(self, **kwargs):
        
        super(VGG16, self).__init__(name='VGG16')
        
        filters = kwargs['filters']
        self.layer1_f = filters[0]
        self.layer2_f = filters[1]
        self.layer3_f = filters[2]
        self.layer4_f = filters[3]
        self.conv = kwargs['conv']
        self.num_classes = kwargs['num_classes']
        self.input_shapes = kwargs['input_shape']
        #print(filters)
        self.conv0 = layers.Conv2D(filters=self.layer1_f, kernel_size=self.conv, input_shape=self.input_shapes, 
                                   activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))
        self.conv1 = layers.Conv2D(filters=self.layer1_f, kernel_size=(self.conv,self.conv), padding='same',activation='relu',
                                   kernel_regularizer=regularizers.l2(0.0001))
        self.maxp1 = layers.MaxPool2D(pool_size=(2,2))
        
        self.conv2 = layers.Conv2D(filters=self.layer2_f, kernel_size=self.conv, padding='same', activation='relu',
                                   kernel_regularizer=regularizers.l2(0.0001))
        self.maxp2 = layers.MaxPool2D(pool_size=(2,2))
        
        self.conv3 = layers.Conv2D(filters=self.layer3_f, kernel_size=self.conv, padding='same', activation='relu',
                                   kernel_regularizer=regularizers.l2(0.0001))
        self.maxp3 = layers.MaxPool2D(pool_size=(2,2))
        
        self.conv4 = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv,padding='same', activation='relu',
                                   kernel_regularizer=regularizers.l2(0.0001))
        self.maxp4 = layers.MaxPool2D(pool_size=(2,2))
        
        self.conv5 = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv, padding='same',activation='relu',
                                   kernel_regularizer=regularizers.l2(0.0001))
        self.maxp5 = layers.MaxPool2D(pool_size=(2,2))
        
        self.fc1 = layers.Dense(512)
        self.fc2 = layers.Dense(512)
        
        self.out = layers.Dense(self.num_classes, activation='softmax')
        
    def call(self, inputs, training=None):
        # first layer
        x = self.conv0(inputs)
        x = self.conv1(x)
        x = self.maxp1(x)
        #print('first layer',x.shape)
        # second layer
        x = self.conv2(x)
        x = layers.Conv2D(filters=self.layer2_f, kernel_size=self.conv, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = self.maxp2(x)
        x = layers.Dropout(0.4)(x)
        
        # third layer
        x = self.conv3(x)
        x = layers.Conv2D(filters=self.layer3_f, kernel_size=self.conv, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.Conv2D(filters=self.layer3_f, kernel_size=self.conv, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = self.maxp3(x)
        x = layers.Dropout(0.3)(x)
        
        # fourth layer
        x = self.conv4(x)
        x = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv,padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv,padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = self.maxp4(x)
        x = layers.Dropout(0.2)(x)
        
        # fifth layer
        x = self.conv5(x)
        x = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv, padding='same',activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.Conv2D(filters=self.layer4_f, kernel_size=self.conv, padding='same',activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = self.maxp5(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Flatten()(x)
        #Fully connected layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = layers.Dropout(0.4)(x)
        
        #output layer
        x = self.out(x)
        return x

In [0]:
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.losses.categorical_crossentropy(labels, logits))

def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    #print('predictions', predictions)
    #print('label', labels)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels ), tf.float32))

In [0]:
def train():
    n_epoch = 20
    train_ds, test_ds = load_data() 
    optimizer = optimizers.Adam(learning_rate=0.0001)
    model = VGG16(**config)
    acc = metrics.CategoricalAccuracy()
    lossfn = tf.keras.losses.CategoricalCrossentropy()
    for epoch in range(n_epoch):
        losses = 0.0
        accuracy = 0.0
        #step = 0
        for step, (x,y) in enumerate(train_ds):
            #print(x.shape)
            #step += 1
            with tf.GradientTape() as tape:
                logits = model(x)
                #y = tf.squeeze(y, axis=1)
                #loss = compute_loss(logits, y)
                loss = lossfn(tf.one_hot(y,10),logits)
                acc.update_state(tf.one_hot(y,10), logits)
                losses = loss
            grads = tape.gradient(loss, model.trainable_variables)
            grads = [tf.clip_by_norm(g, 15) for g in grads]
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            if step%1000 == 0:
                #accuracy = compute_accuracy(logits, tf.cast(y, tf.int64))
                #print(f' accuracy in epoch {epoch} after steps {step} is: {accuracy.numpy()}')
                print(f' accuracy in epoch {epoch} after steps {step} is: {acc.result().numpy()}')
                acc.reset_states()
                #print(f' Loss in epoch {epoch} after steps {step} is: {losses}')
                print(f' Loss in epoch {epoch} after steps {step} is: {losses.numpy()}')
        print(f'loss after epoch {epoch} is {losses.numpy()}')
    print(f'final training accuracy is {acc.result().numpy()}')
    acc.reset_states()  
    #model.save('VGG16', save_format='tf')
    for step, (x, y) in enumerate(test_ds):
        logits = model(x, training=False)
        acc.update_state(tf.one_hot(y,10), logits)
        # if step%1000 == 0:
        #     #y = tf.squeeze(y, axis=1)
        #     #accuracy = compute_accuracy(logits,tf.cast(y, tf.int64))
            
    print(f'test accuracy is {acc.result().numpy()}')
    acc.reset_states()

In [0]:
train()

170500096/170498071 [==============================] - 6s 0us/step
train shape: (50000, 32, 32, 3) (10000, 32, 32, 3)
test shape: (10000, 32, 32, 3) (10000, 1)
 accuracy in epoch 0 after steps 0 is: 0.06689453125
 Loss in epoch 0 after steps 0 is: 2.3025686740875244
loss after epoch 0 is 2.299431324005127
 accuracy in epoch 1 after steps 0 is: 0.09995273500680923
 Loss in epoch 1 after steps 0 is: 2.301731586456299
loss after epoch 1 is 2.3019790649414062
 accuracy in epoch 2 after steps 0 is: 0.09965391457080841
 Loss in epoch 2 after steps 0 is: 2.303251028060913
loss after epoch 2 is 2.304696798324585
 accuracy in epoch 3 after steps 0 is: 0.0990837812423706
 Loss in epoch 3 after steps 0 is: 2.3022027015686035
loss after epoch 3 is 2.2991504669189453
 accuracy in epoch 4 after steps 0 is: 0.1020594909787178
 Loss in epoch 4 after steps 0 is: 2.301455497741699
loss after epoch 4 is 2.302205801010132
 accuracy in epoch 5 after steps 0 is: 0.09839361160993576
 Loss in epoch 5 after st